In [12]:
using ITensors, ITensorMPS
# using LinearAlgebra
# using SparseArrays
# using Plots
# using JLD2

In [13]:
include("src/main.jl")
include("src/observables.jl")
include("src/states.jl")
include("src/doubled_measurements.jl")

forced_doubled_measure_with_val (generic function with 2 methods)

In [14]:
function MPS_update_data(ρ::MPS, observables::Vector{Symbol}, data::Dict{Symbol,Vector{Float64}}, t::Int; ref=false)
    if :Ic in observables
        data[:Ic][t] = Ic2(ρ)
    end
    if :SR in observables
        data[:SR][t] = SR(ρ)
    end
    if :κEA in observables
        data[:κEA][t] = κEA(ρ; ref=ref)
    end
    if :κ2 in observables
        data[:κ2][t] = κ2(ρ; ref=ref)
    end
    if :maxlinkdim in observables
        data[:maxlinkdim][t] = maxlinkdim(ρ)
    end
    return data
end

MPS_update_data (generic function with 1 method)

In [15]:
function circuit(L::Int, T::Int, λ::Float64, δ::Float64, q::Float64; observables=Symbol[], cutoff=1E-8, maxdim=200)
    ρ, sites = ghz(L; ref=true)

    data = Dict([s => zeros(Float64, 2T+2) for s in observables])
    data = MPS_update_data(ρ, observables, data, 1; ref=true)
    data = MPS_update_data(ρ, observables, data, 2; ref=true)

    λzz = δ*(1-λ)
    λx = δ*λ

    Xn = decoherence_layer(sites, PauliX, q, 1:L, L)
    ZZn = decoherence_layer(sites, kron(PauliZ, PauliZ), q, 1:L-1, L)
    bell_state = bell(sites)


    for t in 1:T
        ρ, _, _ = doubled_measure(ρ, PauliX, λx, 1:L; ref=true)
        ρ /= inner(bell_state, ρ)
        truncate!(ρ; cutoff=cutoff, maxdim=maxdim)

        ρ = apply(Xn, ρ)
        ρ /= inner(bell_state, ρ)
        truncate!(ρ; cutoff=cutoff, maxdim=maxdim)

        data = MPS_update_data(ρ, observables, data, 2t+1; ref=true)

        ρ, _, _ = doubled_measure(ρ, kron(PauliZ,PauliZ), λzz, 1:L-1; ref=true)
        ρ /= inner(bell_state, ρ)
        truncate!(ρ; cutoff=cutoff, maxdim=maxdim)

        ρ = apply(ZZn, ρ)
        ρ /= inner(bell_state, ρ)
        truncate!(ρ; cutoff=cutoff, maxdim=maxdim)

        data = MPS_update_data(ρ, observables, data, 2t+2; ref=true)
    end

    return ρ, data
end

circuit (generic function with 1 method)

In [ ]:
function sample(L::Int, T::Int, Δ::Float64, δ::Float64, q::Float64, samples::Int; observables=Symbol[], cutoff=1E-8, maxdim=200)
    mean_data = Dict([s => zeros(Float64, 2T+2) for s in observables])
    var_data = Dict([s => zeros(Float64, 2T+2) for s in observables])
    for _ in 1:samples
        _, sample_data = circuit(L, T, Δ, δ, q; observables=observables, cutoff=cutoff, maxdim=maxdim)
        for observable in observables
            mean_data[observable] += sample_data[observable]
            var_data[observable] += sample_data[observable].^2
        end
    end
    for observable in observables
        mean_data[observable] /= samples
        var_data[observable] /= samples
    end
    return mean_data, var_data
end

sample (generic function with 1 method)

In [21]:
sample(4, 4, 0.1, 0.1, 0.1, 10; observables=[:Ic, :SR, :κEA, :κ2, :maxlinkdim])

(Dict(:κEA => [4.0, 4.0, 1.5025118175309566, 1.4803392103935515, 1.080457259368843, 1.100402140647492, 1.0168220150253966, 1.040203350035119, 1.0067346074025447, 1.0217289136989898], :SR => [1.0, 1.0, 0.999999999999982, 1.0000001699601513, 1.0000008581149509, 1.0000007283095997, 1.0000005248780854, 1.0000002110036152, 1.0000000090670131, 0.9999994583819747], :maxlinkdim => [4.0, 4.0, 8.0, 8.0, 12.3, 12.0, 15.6, 14.1, 16.0, 15.7], :Ic => [1.0, 1.0, 0.9966309188631455, 0.9952438750877171, 0.9494353379662916, 0.9469969717256232, 0.8278172724160605, 0.8041863582400589, 0.6468841210436524, 0.6633992047269764], :κ2 => [4.0, 4.0, 3.9970747263243895, 3.998306956340211, 3.995153096415753, 3.997261525219153, 3.9928945275195327, 3.9959956979884543, 3.991349993105382, 3.9953050775888976]), Dict(:κEA => [16.0, 16.0, 2.257541761820178, 2.2173193781242726, 1.1681130703800506, 1.2141638018673462, 1.0340191936855008, 1.082876687165492, 1.0135385131270045, 1.0444296159873292], :SR => [1.0, 1.0, 0.999999

In [17]:
# data = Dict{Tuple{Int, Int, Float64, Float64, Float64},Tuple{Int,Dict}}()

In [ ]:
# observables = Symbol[:κEA, :κ2, :Ic, :SR, :maxlinkdim]
# samples = 250
# δ = 0.7

# # observables = Symbol[:maxlinkdim]

# for q in [0.1]
#     for λ in 0.0:0.1:1.0
#         for L in 4:4:12
#             T = 2L
#             data[(L,T,λ,δ,q)] = (samples, sample(L, T, λ, δ, q, samples; observables=observables))
#         end
#     end
# end

In [ ]:
# jldsave("25_07_22.jld2"; data)

In [ ]:
data

Dict{Tuple{Int64, Int64, Float64, Float64, Float64}, Tuple{Int64, Dict}} with 33 entries:
  (8, 16, 0.6, 0.7, 0.1)  => (250, Dict(:κEA=>[8.0, 8.0, 1.06778, 1.57057, 1.00…
  (12, 24, 0.0, 0.7, 0.1) => (250, Dict(:κEA=>[12.0, 12.0, 2.84549, 9.76532, 2.…
  (8, 16, 0.1, 0.7, 0.1)  => (250, Dict(:κEA=>[8.0, 8.0, 2.08585, 5.74068, 1.73…
  (12, 24, 0.3, 0.7, 0.1) => (250, Dict(:κEA=>[12.0, 12.0, 1.91091, 6.15861, 1.…
  (4, 8, 0.4, 0.7, 0.1)   => (250, Dict(:κEA=>[4.0, 4.0, 1.1432, 2.13562, 1.056…
  (8, 16, 1.0, 0.7, 0.1)  => (250, Dict(:κEA=>[8.0, 8.0, 1.00022, 1.00022, 1.00…
  (4, 8, 0.6, 0.7, 0.1)   => (250, Dict(:κEA=>[4.0, 4.0, 1.02905, 1.37289, 1.00…
  (4, 8, 0.1, 0.7, 0.1)   => (250, Dict(:κEA=>[4.0, 4.0, 1.46536, 3.11431, 1.32…
  (8, 16, 0.2, 0.7, 0.1)  => (250, Dict(:κEA=>[8.0, 8.0, 1.85823, 5.36032, 1.53…
  (8, 16, 0.7, 0.7, 0.1)  => (250, Dict(:κEA=>[8.0, 8.0, 1.02335, 1.20661, 1.00…
  (8, 16, 0.8, 0.7, 0.1)  => (250, Dict(:κEA=>[8.0, 8.0, 1.00653, 1.04817, 1.00…
  (12, 24, 0.9, 0.7